In [ ]:
import sys
sys.path.append("..") #this is to add the avobe folder to the package directory
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import os
import shapely
from nexustool.gis_tools import download_data, create_time_data, get_area_share, get_zonal_stats
from nexustool.weap_tools import reproject_raster, sample_raster

## Downloading solar irradiation and water table depth data

In [ ]:
url = 'https://biogeo.ucdavis.edu/data/worldclim/v2.1/base/wc2.1_30s_srad.zip'
file_path = os.path.join('data', 'gis', 'srad', 'wc2.1_30s_srad.zip')
download_data(url, file_path)

In [ ]:
url = 'https://souss-massa-dev.s3.us-east-2.amazonaws.com/post_build/Africa_model_wtd_v2.nc'
file_path = os.path.join('data', 'gis', 'wtd', 'Africa_model_wtd_v2.nc')
download_data(url, file_path)

## Reading the input data

In [ ]:
data_folder = os.path.join('data')
demand_path = os.path.join(data_folder, 'gis', 'processed layers', 'demand_points.gpkg')
cropland_path = os.path.join(data_folder, 'gis', 'cropland', 'cropland.gz')

crop_df = pd.read_csv(cropland_path, encoding='utf-8')
geometry = crop_df['WKT'].map(shapely.wkt.loads)
cropland = gpd.GeoDataFrame(crop_df.drop(columns=['WKT']), crs="EPSG:26192", geometry=geometry)
provinces = gpd.read_file(os.path.join('data', 'gis', 'admin', 'provinces.gpkg'), encoding='utf-8')

## Convert coordenate reference system (crs)

In [ ]:
MerchidSudMoroc = 26192
for gdf in [provinces, demand_sites]:
    gdf.to_crs(epsg=MerchidSudMoroc, inplace=True)

cropland = cropland.loc[cropland.area_m2>=100] #choose

## Solar irradiation zonal statistics
Loops through the 12 months of the year and gets the mean solar irradiation of each month within each cropland polygon

In [ ]:
cropland.to_crs(epsg=4326, inplace=True)
for month in range(1, 13):
    cropland = get_zonal_stats(cropland, 
                               os.path.join('data', 'gis', 'srad', 
                                            f'wc2.1_30s_srad_{str(month).zfill(2)}.tif'), 
                               ['mean'], all_touched=True).rename(columns={'mean': f'srad{month}'})

## Water table depth zonal statistics

In [ ]:
cropland.crs = 4326
cropland = get_zonal_stats(cropland, 
                           os.path.join('data', 'gis', 'wtd', 
                                        'Africa_model_wtd_v2.nc'), 
                           ['mean'], all_touched=True).rename(columns={'mean': 'wtd'})

In [ ]:
cropland.crs = 4326
cropland.to_crs(epsg=MerchidSudMoroc, inplace=True)

In [ ]:
cropland.head()

## Visualizing the solar irriadiation in cropland areas

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 12))

divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.1)

provinces.plot(color='white', edgecolor='black', ax=ax)
cropland.plot(column='srad1', ax=ax, cmap='OrRd', legend=True, cax=cax)

## Creating time series data

In [ ]:
df_cropland = create_time_data(cropland, 2019, 2050)

In [ ]:
df_cropland.head()

## Calculating the area share of each croplan area within each province

In [ ]:
cropland_temp = cropland.copy()
cropland_temp.loc[cropland_temp['province']=='Inezgane-Aït Melloul', 'province'] = 'Taroudannt' #Including Inezgane-Aït Melloul irrigated area into results from Taroudant due to lack of data for the former
df_cropland['area_share'] = df_cropland['Demand point'].map(get_area_share(cropland_temp, 'province', 'area_m2'))

## Saving the results

In [ ]:
os.makedirs(os.path.join('data', 'cropland'), exist_ok = True)
df_cropland.to_csv(os.path.join('data', 'cropland', 'cropland.gz'), index=False)